### python dataframe 获得 列名columns 和行名称 index


In [3]:
import numpy as np
import pandas as pd
# 用list的数据创建dataframe：
a = [['2', '1.2', '4.2'], ['0', '10', '0.3'], ['1', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
print (df)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print(df.columns.values.tolist())    # 列名称
print(df._stat_axis.values.tolist()) # 行名称

  one  two three
0   2  1.2   4.2
1   0   10   0.3
2   1    5     0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
['one', 'two', 'three']
[0, 1, 2]


## DataFrame的创建
### 创建一个空的dataframe 


In [8]:
import pandas as pd
print(pd.DataFrame(columns={"a":"","b":"","c":""},index=[0]))

print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
# 也可以直接:
test_pd = pd.DataFrame()
test_pd = test_pd.append({ 'row1' : [1,2,3,4], 'row2' : ['a' , 'b' , 'c' , 'd'] }, ignore_index= True)
print(test_pd)

a    b    c
0  NaN  NaN  NaN
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
           row1          row2
0  [1, 2, 3, 4]  [a, b, c, d]



## 数据合并，连接（merge,join,concat)
### merge  通过键拼接列
pandas提供了一个类似于关系数据库的连接(join)操作的方法merage,可以根据一个或多个键将不同DataFrame中的行连接起来

语法如下：

```
merge(left, right, how='inner', on=None, left_on=None, right_on=None,
      left_index=False, right_index=False, sort=True,
      suffixes=('_x', '_y'), copy=True, indicator=False)
```

用于通过一个或多个键将两个数据集的行连接起来，类似于 SQL 中的 JOIN。

该函数的典型应用场景是：针对同一个主键存在两张包含不同字段的表，现在我们想把他们整合到一张表里。在此典型情况下，结果集的行数并没有增加，列数则为两个元数据的列数和减去连接键的数量。

1. on=None 用于显示指定列名（键名），如果该列在两个对象上的列名不同，则可以通过 left_on=None, right_on=None 来分别指定。或者想直接使用行索引作为连接键的话，就将 left_index=False, right_index=False 设为 True。
2. how='inner' 参数指的是当左右两个对象中存在不重合的键时，取结果的方式：inner 代表交集；outer 代表并集；left 和 right 分别为取一边。
3. suffixes=('_x','_y') 指的是当左右对象中存在除连接键外的同名列时，结果集中的区分方式，可以各加一个小尾巴。
4. 对于多对多连接，结果采用的是行的笛卡尔积。

参数说明：
* left与right：两个不同的DataFrame
* how：指的是合并(连接)的方式有inner(内连接),left(左外连接),right(右外连接),outer(全外连接);默认为inner
* on : 指的是用于连接的列索引名称。必须存在右右两个DataFrame对象中，如果没有指定且其他参数也未指定则以两个DataFrame的列名交集做为连接键
* left_on：左则DataFrame中用作连接键的列名;这个参数中左右列名不相同，但代表的含义相同时非常有用。
* right_on：右则DataFrame中用作 连接键的列名
* left_index：使用左则DataFrame中的行索引做为连接键
* right_index：使用右则DataFrame中的行索引做为连接键
* sort：默认为True，将合并的数据进行排序。在大多数情况下设置为False可以提高性能
* suffixes：字符串值组成的元组，用于指定当左右DataFrame存在相同列名时在列名后面附加的后缀名称，默认为('_x','_y')
* copy：默认为True,总是将数据复制到数据结构中；大多数情况下设置为False可以提高性能
* indicator：在 0.17.0中还增加了一个显示合并数据中来源情况；如只来自己于左边(left_only)、两者(both)

##### merge 一些特性展示
1. 默认以重叠的列名当做链接键.

In [4]:
from pandas import DataFrame
import pandas as pd
p = print
df1=DataFrame({'key':['a','b','b'],'data1':range(3)})
df2=DataFrame({'key':['a','b','c'],'data2':range(3)})

p(pd.merge(df1, df2)) #没有指定连接键，默认用重叠列名，没有指定连接方式
p(df1)
p(df2)

  key  data1  data2
0   a      0      0
1   b      1      1
2   b      2      1
  key  data1
0   a      0
1   b      1
2   b      2
  key  data2
0   a      0
1   b      1
2   c      2


In [10]:
# 2.默认做inner连接（取key的交集），连接方式还有（left,right,outer)，制定连接方式加参数：how=''
p(pd.merge(df2,df1)) # 笛卡尔积
p(pd.merge(df2,df1,how='left'))    #通过how，指定连接方式
p(pd.merge(df1,df2,how='left'))

key  data2  data1
0   a      0      0
1   b      1      1
2   b      1      2
  key  data2  data1
0   a      0    0.0
1   b      1    1.0
2   b      1    2.0
3   c      2    NaN
  key  data1  data2
0   a      0      0
1   b      1      1
2   b      2      1


In [9]:
# 3.多键连接时将连接键组成列表传入，例：pd.merge(df1,df2,on=['key1','key2']

right=DataFrame({'key1':['foo','foo','bar','bar'],
         'key2':['one','one','one','two'],
         'lval':[4,5,6,7]})
 
left=DataFrame({'key1':['foo','foo','bar'],
         'key2':['one','two','one'],
         'lval':[1,2,3]})
 
pd.merge(left,right,on=['key1','key2'],how='outer')  #传出数组

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [12]:
#4.如果两个对象的列名不同，可以分别指定，例：
df3=DataFrame({'key3':['foo','foo','bar','bar'], #将上面的right的key 改了名字
         'key4':['one','one','one','two'],
         'lval':[4,5,6,7]})
 
pd.merge(left,df3,left_on='key1',right_on='key3')  #键名不同的连接

,key1,key2,lval_x,key3,key4,lval_y
0,foo,one,1,foo,one,4
1,foo,one,1,foo,one,5
2,foo,two,2,foo,one,4
3,foo,two,2,foo,one,5
4,bar,one,3,bar,one,6
5,bar,one,3,bar,two,7


In [16]:
import pandas as pd

left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})
result = pd.merge(left, right, on='key')

# on参数传递的key作为连接键
result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


In [18]:
# 传入的on的参数是列表：
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                      'key2': ['K0', 'K1', 'K0', 'K1'],
                         'A': ['A0', 'A1', 'A2', 'A3'],
                         'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                         'C': ['C0', 'C1', 'C2', 'C3'],
                         'D': ['D0', 'D1', 'D2', 'D3']})

result = pd.merge(left, right, on=['key1', 'key2'])
# 同时传入两个Key，此时会进行以['key1','key2']列表的形式进行对应，left的keys列表是：[['K0', 'K0'],['K0', 'K1'],['K1', 'K0'],['K2', 'K1']],left的keys列表是：[['K0', 'K0'],['K1', 'K0'],['K1', 'K0'],['K2', 'K0']]，因此会有1个['K0', 'K0']、2个['K1', 'K0']对应。

result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [19]:
# Merge method : 如果组合键没有出现在左表或右表中，则连接表中的值将为NA。
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
# Use keys from left frame only
result


,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


In [20]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
# Use keys from right frame only
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [21]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
# Use intersection of keys from both frames 
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


In [22]:
left = pd.DataFrame({'A' : [1,2], 'B' : [2, 2]})
right = pd.DataFrame({'A' : [4,5,6], 'B': [2,2,2]})
result = pd.merge(left, right, on='B', how='outer')
result

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


 4. 传入indicator参数merge接受参数指示符。 如果为True，则将名为_merge的Categorical类型列添加到具有值的输出对象：

In [23]:
df1 = pd.DataFrame({'col1': [0, 1], 'col_left':['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2],'col_right':[2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [24]:
# 指标参数也将接受字符串参数，在这种情况下，指标函数将使用传递的字符串的值作为指标列的名称。
pd.merge(df1, df2, on='col1', how='outer', indicator='indicator_column')

,col1,col_left,col_right,indicator_column
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


5. 以index为链接键
需要同时设置left_index= True 和 right_index= True，或者left_index设置的同时，right_on指定某个Key。总的来说就是需要指定left、right链接的键，可以同时是key、index或者混合使用。

In [25]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
   ....:                      'B': ['B0', 'B1', 'B2']},
   ....:                      index=['K0', 'K1', 'K2'])
   ....: 
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
   ....:                       'D': ['D0', 'D2', 'D3']},
   ....:                       index=['K0', 'K2', 'K3'])
   ....: 

# 只有K0、K2有对应的值
pd.merge(left,right,how= 'inner',left_index=True,right_index=True)

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [26]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                   'key': ['K0', 'K1', 'K0', 'K1']})


right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                    index=['K0', 'K1'])


result = pd.merge(left, right, left_on='key', right_index=True, how='left', sort=False)
#  left_on='key', right_index=True
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


6.sort对链接的键值进行排序：

In [28]:
# 紧接着上一例，设置sort= True
result = pd.merge(left, right, left_on='key', right_index=True, how='left', sort=True)

result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
2,A2,B2,K0,C0,D0
1,A1,B1,K1,C1,D1
3,A3,B3,K1,C1,D1


* 5.以索引当做连接键，使用参数left_index=true,right_index=True （最好使用join）

### join 拼接列，主要用于索引上的合并

join方法提供了一个简便的方法用于将两个DataFrame中的不同的列索引合并成为一个DataFrame
```
join(self, other, on=None, how='left', lsuffix='', rsuffix='',sort=False):
```
其中参数的意义与merge方法基本相同,只是join方法默认为左外连接how=left

* 1.默认按索引合并，可以合并相同或相似的索引，不管他们有没有重叠列。

* 2.可以连接多个DataFrame

* 3.可以连接除索引外的其他列

* 4.连接方式用参数how控制

* 5.通过lsuffix='', rsuffix='' 区分相同列名的列


### concat  可以沿着一条轴将多个对象堆叠到一起
concat方法相当于数据库中的全连接(UNION ALL),可以指定按某个轴进行连接,也可以指定连接的方式join(outer,inner 只有这两种)。与数据库不同的时concat不会去重，要达到去重的效果可以使用drop_duplicates方法

```
 concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
           keys=None, levels=None, names=None, verify_integrity=False, copy=True):
```

轴向连接 pd.concat() 就是单纯地把两个表拼在一起，这个过程也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。因此可以想见，这个函数的关键参数应该是 axis，用于指定连接的轴向。

在默认的 axis=0 情况下，pd.concat([obj1,obj2]) 函数的效果与 obj1.append(obj2) 是相同的；

而在 axis=1 的情况下，pd.concat([df1,df2],axis=1) 的效果与 pd.merge(df1,df2,left_index=True,right_index=True,how='outer') 是相同的。

可以理解为 concat 函数使用索引作为“连接键”。
本函数的全部参数为：
```
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False)。
```
objs 就是需要连接的对象集合，一般是列表或字典；

axis=0 是连接轴向join='outer' 参数作用于当另一条轴的 index 不重叠的时候，只有 'inner' 和 'outer' 可选（顺带展示 ignore_index=True 的用法）

 

concat 一些特点：

1. 作用于Series时，如果在axis=0时，类似union。axis=1 时，组成一个DataFrame，索引是union后的，列是类似join后的结果。

2. 通过参数join_axes=[] 指定自定义索引。

3. 通过参数keys=[] 创建层次化索引

4. 通过参数ignore_index=True 重建索引。

In [14]:
import numpy as np
df1=DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])

df2=DataFrame(np.random.randn(2,3),columns=['b','d','a'])
 
pd.concat([df1,df2])

,a,b,c,d
0,0.462355,-2.349735,0.958933,-1.584726
1,-0.357857,0.448201,-0.342158,1.237455
2,0.545077,-1.715210,0.219724,-0.929223
0,-0.433747,0.955138,NaN,1.294303
1,0.880044,-1.491056,NaN,0.304303


In [15]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,0.462355,-2.349735,0.958933,-1.584726
1,-0.357857,0.448201,-0.342158,1.237455
2,0.545077,-1.715210,0.219724,-0.929223
3,-0.433747,0.955138,NaN,1.294303
4,0.880044,-1.491056,NaN,0.304303


#### DataFrame的append方法详细介绍

DataFrame.append(other, ignore_index=False, verify_integrity=False, sort=None)

功能说明：向dataframe对象中添加新的行，如果添加的列名不在dataframe对象中，将会被当作新的列进行添加

* other：DataFrame、series、dict、list这样的数据结构
* ignore_index：默认值为False，如果为True则不使用index标签
* verify_integrity ：默认值为False，如果为True当创建相同的index时会抛出ValueError的异常
* sort：boolean，默认是None，该属性在pandas的0.23.0的版本才存在。

In [1]:
import pandas as pd

data = pd.DataFrame()
a = {'x':1, 'y':2}
data = data.append(a, ignore_index= True)
print(data)

x    y
0  1.0  2.0


#### append添加series
如果不添加ignore_index=True，会报错提示TypeError: Can only append a Series if ignore_index=True or if the Series has a name，如果不添加ignore_index=True，也可以改成以下代码
* 注意：当dataframe使用append方法添加series的时候，必须要设置name，设置name名称将会作为index的name。

In [2]:
import pandas as pd

data = pd.DataFrame()
series = pd.Series({"x":1,"y":2},name="a")
data = data.append(series)
print(data)

x    y
a  1.0  2.0


#### append添加list

如果list是一维的，则是以列的形式来进行添加，如果list是二维的则是以行的形式进行添加的，如果是三维的则只添加一个值


In [3]:
data = pd.DataFrame()
a = [1,2,3]
data = data.append(a)
print(data)

0
0  1
1  2
2  3


In [4]:
data = pd.DataFrame()
a = [[[1,2,3]]]
data = data.append(a)
print(data)

0
0  [1, 2, 3]


注意：在多次使用append方法追加数据的时候，可能会出现相同的index

In [5]:
data = pd.DataFrame()
a = [[1,2,3],[4,5,6]]
data = data.append(a)
a = [[7,8,9],[10,11,12]]
data = data.append(a)
print(data)

0   1   2
0   1   2   3
1   4   5   6
0   7   8   9
1  10  11  12


如果想要添加的index不出现重复的情况，可以通过设置ignore_index=True来避免

In [6]:
data = pd.DataFrame()
a = [[1,2,3],[4,5,6]]
data = data.append(a,ignore_index=True)
a = [[7,8,9],[10,11,12]]
data = data.append(a,ignore_index=True)
print(data)

0   1   2
0   1   2   3
1   4   5   6
2   7   8   9
3  10  11  12


### Pandas —— sort_value( )、sort_index( )排序

### pandas按行按列遍历Dataframe的几种方式

* iterrows(): 按行遍历，将DataFrame的每一行迭代为(index, Series)对，可以通过row\[name\]对元素进行访问。
* itertuples(): 按行遍历，将DataFrame的每一行迭代为元祖，可以通过row\[name\]对元素进行访问，比iterrows()效率高。
* iteritems():按列遍历，将DataFrame的每一列迭代为(列名, Series)对，可以通过row\[index\]对元素进行访问。

In [5]:
import pandas as pd

inp = [{'c1':10, 'c2':100}, {'c1':11, 'c2':110}, {'c1':12, 'c2':123}]
df = pd.DataFrame(inp)

print(df)

   c1   c2
0  10  100
1  11  110
2  12  123


In [10]:
for index, row in df.iterrows():
    print(row) # 输出每行的索引值

print(type(row))

c1     10
c2    100
Name: 0, dtype: int64
c1     11
c2    110
Name: 1, dtype: int64
c1     12
c2    123
Name: 2, dtype: int64
<class 'pandas.core.series.Series'>


#### 按行遍历itertuples():

In [9]:
for row in df.itertuples():
    print(getattr(row, 'c1'), getattr(row, 'c2')) # 输出每一行

10 100
11 110
12 123


#### 按列遍历iteritems():

In [12]:
for index, row in df.iteritems():
    #print(index) # 输出列名
    print(row)


0    10
1    11
2    12
Name: c1, dtype: int64
0    100
1    110
2    123
Name: c2, dtype: int64


In [14]:
for  index, row in df.iteritems():
    print(row[0], row[1], row[2]) # 输出各列

10 11 12
100 110 123


## DataFrame 对每一行做 lambda 处理

In [3]:
import pandas as pd
def ff(x):
    if x["1"] < 4:
        return x["1"]
    return x["2"]


dd = pd.DataFrame()
df = pd.DataFrame({"1": [1, 2, 3,4], "2": [10, 11, 12,13]})
print(df)

print("---------------")
rr = df.apply(lambda x: ff(x), axis=1)
df["calc"] = rr

print(df)

   1   2
0  1  10
1  2  11
2  3  12
3  4  13
---------------
   1   2  calc
0  1  10     1
1  2  11     2
2  3  12     3
3  4  13    13


## 删除DataFrame中值全为NaN或者包含有NaN的列或行

如果存在以下DataFrame
     年龄      性别        手机号

0    2             男           NaN

1    3             女           NaN

2    4             NaN       NaN

 
删除NaN所在的行：
删除表中全部为NaN的行

df.dropna(axis=0,how='all')  

删除表中含有任何NaN的行

df.dropna(axis=0,how='any') #drop all rows that have any NaN values

删除NaN所在的列：

删除表中全部为NaN的列
df.dropna(axis=1,how='all') 

删除表中含有任何NaN的列
df.dropna(axis=1,how='any') #drop all rows that have any NaN values